# How to train a tensorflow model from scratch

## Preparation of libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import requests
import time
import os
import re

# Personal access data
import authInfo

Github repo path for loading training images and test images

In [2]:
# Github Repository Information
repoOwner = 'yanglinfang'
repoName = 'npo_tech_wildlife_camera'
trainPath = 'dataset/train_set/wild/'
testPath = 'dataset/test_set/wild/'
myToken = authInfo.gitHubToken

# GitHub API endpoint to list contents of the folder
trainImageUrl = f'https://api.github.com/repos/{repoOwner}/{repoName}/contents/{trainPath}'
testImageUrl = f'https://api.github.com/repos/{repoOwner}/{repoName}/contents/{testPath}'

# OAuth Token used for more data limit
githubHeaders = {
    'Authorization': f'token {myToken}'
}

# Retry times when failed to download image
maxRetry = 10


Convert image to same shape for model training

In [3]:
# Define the common width
common_width = 256
common_height = 256

Variables used in later model training

In [4]:
# list of numpy that stores train image in numpy format
trainFeatures = []
trainLabels = [0] * 393
for i in range(len(trainLabels)):
    if i >= 343 and i <= 392:
        trainLabels[i] = 1

# list of numpy that stores test image in numpy format
testFeatures = []
testLabels = [0] * 160
for i in range(len(testLabels)):
    if i >= 140 and i <= 159:
        testLabels[i] = 1

# number of times model trains itself
numEpochs = 10

# how many files being trained each iteration
batchSize = 32

Get Image from train and test to be numpy array

In [6]:
def image_to_numpy(githubApiUrl, maxRetry = 10, numpyList = []):
    # Send HTTP GET request
    response = requests.get(githubApiUrl, headers=githubHeaders)

    #Check if request was successful(status code 200)
    if response.status_code == 200:

        # Turn the folder into a file 
        folderContents = response.json()

        # Sort filenames based on numeric values
        sortedFolder = sorted(folderContents, key=lambda x: extract_numeric_name(x['name']))

        # Loop through contents of folder
        for item in sortedFolder:
            if item['type'] == 'file':
                fileName = item['name']
                imageUrl = item['download_url']
                # print(fileName)
                for retry in range(maxRetry):
                    try:
                        # Send an HTTP GET request to load the image
                        imageResponse = requests.get(imageUrl, timeout=10)

                        if imageResponse.status_code == 200:
                            imageNp = np.frombuffer(imageResponse.content, dtype = np.uint8)
                            image = cv2.imdecode(imageNp, cv2.IMREAD_COLOR)
                            resizedImage = cv2.resize(image, (common_width, common_height))
                            numpyList.append(resizedImage)
                            break
                    except requests.exceptions.RequestException as e:
                        time.sleep(1)
                else:
                    print(f'Failed to downloadimage {fileName} after {maxRetry} retries')
    else:
        print(f'Failed to fetch folder: Status Code {response.status_code}')

Some helper function for sorting data in the needed order

In [7]:
# code to sort file in numeric order
def extract_numeric_name(file):
    match = re.search(r'\d+', file)
    if match:
        return int(match.group())
    return -1

Execute to get the images from cloud then convert and add into the numpy array

In [8]:
image_to_numpy(trainImageUrl, maxRetry, trainFeatures)
image_to_numpy(testImageUrl, maxRetry, testFeatures)

## Create Model

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(256, 256, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Flatten(),  # Add a flattening layer to convert the 4D input to 1D
    tf.keras.layers.Dense(2, activation='softmax')
])


## Compile Model

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Train Model

In [11]:
# Convert train lists to NumPy arrays
trainFeatures_np = np.array(trainFeatures)
trainLabels_np = np.array(trainLabels)

# Convert test list to Numpy arrrays
testFeatures_np = np.array(testFeatures)
testLabels_np = np.array(testLabels)

In [13]:
model.fit(trainFeatures_np, trainLabels_np, epochs=numEpochs, batch_size=batchSize)

Epoch 1/10
13/13 [==============================] - 35s 2s/step - loss: 21000.2734 - accuracy: 0.7684
Epoch 2/10
13/13 [==============================] - 26s 2s/step - loss: 3636.5713 - accuracy: 0.7863
Epoch 3/10
13/13 [==============================] - 26s 2s/step - loss: 871.8842 - accuracy: 0.8575
Epoch 4/10
13/13 [==============================] - 26s 2s/step - loss: 141.9894 - accuracy: 0.8473
Epoch 5/10
13/13 [==============================] - 26s 2s/step - loss: 67.2858 - accuracy: 0.8601
Epoch 6/10
13/13 [==============================] - 26s 2s/step - loss: 18.9307 - accuracy: 0.9313
Epoch 7/10
13/13 [==============================] - 26s 2s/step - loss: 45.4250 - accuracy: 0.8753
Epoch 8/10
13/13 [==============================] - 26s 2s/step - loss: 17.7101 - accuracy: 0.9008
Epoch 9/10
13/13 [==============================] - 26s 2s/step - loss: 14.6903 - accuracy: 0.9415
Epoch 10/10
13/13 [==============================] - 26s 2s/step - loss: 18.4738 - accuracy: 0.9389


## Evaluate Model

In [15]:
test_loss, test_accuracy = model.evaluate(testFeatures_np, testLabels_np)
print(f"Test accuracy: {test_accuracy}")

5/5 [==============================] - 5s 896ms/step - loss: 62.8635 - accuracy: 0.8938
Test accuracy: 0.893750011920929


In [16]:
# Use model.predict to obtain predictions for the test dataset
predictions = model.predict(testFeatures_np)

# The predictions are in the form of class probabilities; you can get the predicted class (0 or 1) for each image
predicted_labels = np.argmax(predictions, axis=1)

# Compare predicted_labels with the actual test labels to identify correct and incorrect classifications
correctly_classified_indices = np.where(predicted_labels == testLabels_np)[0]
incorrectly_classified_indices = np.where(predicted_labels != testLabels_np)[0]

# Access the corresponding test images using the indices
correctly_classified_images = [testFeatures_np[i] for i in correctly_classified_indices]
incorrectly_classified_images = [testFeatures_np[i] for i in incorrectly_classified_indices]

5/5 [==============================] - 7s 1s/step


In [20]:
# Calculate the number of correctly classified images
num_correctly_classified = len(correctly_classified_images)

# Total number of test images
num_total_images = len(testLabels_np)

# Create a string summary
summary = f"Correctly Classified: {num_correctly_classified}/{num_total_images}"

# Print the summary
print(summary)

Correctly Classified: 143/160


## Save model for future use

In [22]:
# Legacy way
model.save('boar_recognition.h5')
# Keras format
model.save('boar_recognition.keras')

d:\Programming\python\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
